# Сравнительный анализ активности читателей Яндекс.Книг (СПб vs Москва) и оценка эффективности новой версии интерфейса интернет-магазина BitMotion Kit на основе A/B-тестирования

## Краткое описание:

Проект состоит из двух ключевых частей:

Сравнительный анализ активности читателей Яндекс.Книг в Санкт-Петербурге и Москве – проверить, действительно ли пользователи из Санкт-Петербурга проводят в среднем больше времени за чтением и прослушиванием книг в приложении, чем пользователи из Москвы.

Анализ результатов A/B-тестирования новой версии интерфейса интернет-магазина BitMotion Kit – оценка эффективности изменений в интерфейсе на основе ключевых метрик (конверсия, средний чек, отказы и др.).

## Цель:
Выявить закономерности в поведении пользователей Яндекс.Книг и определить, насколько новая версия интерфейса BitMotion Kit улучшает пользовательский опыт и бизнес-показатели.

## Задачи:

**Для анализа Яндекс.Книг**:

Сравнить количество наблюдений, среднее значение, дисперсию и другие характеристики распределения времени активности между пользователями двух городов.

Провести двусторонний t-тест для независимых выборок, чтобы определить, существует ли статистически значимое различие между средним временем активности пользователей из разных городов.

**Для A/B-теста BitMotion Kit**:

Провести статистический анализ результатов теста (A/B-группы).

Сформулировать рекомендации по внедрению изменений.

## Итог:
Проект объединяет маркетинговый и UX-анализ, помогая оптимизировать сервисы на основе данных о поведении пользователей.

## **Часть 1.** Сравнительный анализ активности читателей Яндекс.Книг СПб vs Москва




2. Сравнить количество наблюдений, среднее значение, дисперсию и другие характеристики распределения времени активности между пользователями двух городов.

3. Провести двусторонний t-тест для независимых выборок, чтобы определить, существует ли статистически значимое различие между средним временем активности пользователей из разных городов.

### Описание данных

Таблицы этого проекта содержат данные о чтении и прослушивании контента в сервисе Яндекс Книги, которые включают информацию о пользователях, платформах, времени, длительности сессий и типах контента. Данные представлены за период с 1 сентября по 11 декабря 2024 года. В вашем распоряжении будет несколько таблиц.

<b>Таблица bookmate.audition содержит данные об активности пользователей и состоит из следующих полей:</b>

•	audition_id — уникальный идентификатор сессии чтения или прослушивания;

•	puid — идентификатор пользователя;

•	usage_platform_ru — название платформы, с помощью которой пользователь слушал контент;

•	msk_business_dt_str — дата события в формате строки (московское время);

•	app_version — версия приложения, которая использовалась для чтения или прослушивания;

•	adult_content_flg — был ли это контент для взрослых: True или False;

•	hours — длительность чтения или прослушивания в часах;

•	hours_sessions_long — продолжительность длинных сессий чтения или прослушивания в часах;

•	kids_content_flg — был ли это детский контент: True или False;

•	main_content_id — идентификатор основного контента;

•	usage_geo_id — идентификатор географического местоположения.

<b>Таблица bookmate.content содержит данные о контенте и состоит из следующих полей:</b>

•	main_content_id — идентификатор основного контента;

•	main_author_id — идентификатор основного автора контента;

•	main_content_type — тип контента;

•	main_content_name— название контента;

•	main_content_duration_hours — длительность контента в часах;

•	published_topic_title_list — список жанров контента.

<b> Таблица bookmate.author содержит данные об авторах контента и состоит из следующих полей:</b>

•	main_author_id — идентификатор основного автора контента;

•	main_author_name — имя основного автора контента.

<b>Таблица bookmate.geo содержит данные о местоположении и состоит из следующих полей:</b>

•	usage_geo_id — идентификатор географического положения;

•	usage_geo_id_name — город или регион географического положения;

•	usage_country_name — страна географического положения.


### 1. Загрузка данных и знакомство с ними

In [1]:
# Импортируем необходимые библиотеки
import pandas as pd
import numpy as np

# Устанавливаем недостающие библиотеки через try-except
try:
    from scipy import stats
except ImportError:
    !pip install scipy
    from scipy import stats

try:
    from scipy.stats import norm
except ImportError:
    !pip install scipy
    from scipy.stats import norm

result = stats.ttest_ind

In [2]:
# Загружаем базу данных
df = pd.read_csv('https://code.s3.yandex.net/datasets/yandex_knigi_data.csv')

In [3]:
# Выводим датасет на экран для анализа 
display(df)
df.info()

,Unnamed: 0,city,puid,hours
0,0,Москва,9668,26.167776
1,1,Москва,16598,82.111217
2,2,Москва,80401,4.656906
3,3,Москва,140205,1.840556
4,4,Москва,248755,151.326434
...,...,...,...,...
8779,8779,Санкт-Петербург,1130000028554332,4.107774
8780,8780,Санкт-Петербург,1130000030307246,45.069222
8781,8781,Санкт-Петербург,1130000038726322,0.211944
8782,8782,Санкт-Петербург,1130000047892100,4.311841


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8784 entries, 0 to 8783
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  8784 non-null   int64  
 1   city        8784 non-null   object 
 2   puid        8784 non-null   int64  
 3   hours       8784 non-null   float64
dtypes: float64(1), int64(2), object(1)
memory usage: 274.6+ KB


Краткий вывод по данным:

Набор данных состоит из 8784 записей с четырьмя столбцами:

<b>Unnamed:0</b>: выглядит как технический индекс, скорее всего автоматически созданный системой при загрузке файла. Его рекомендуется удалить, поскольку такой столбец излишне увеличивает размер датасета и затрудняет восприятие.

<b>city:</b> название города хранится в виде строкового типа, и это хорошо. Однако стоит убедиться, что оно приведено к единому формату (отсутствие пробелов перед названием и после него).

<b>puid:</b> уникальный идентификатор пользователя (целое число).

<b>hours:</b> активность пользователей в часах (десятичные числа).

Все данные заполнены, отсутствуют пропуски. Типы данных верны и подходят для дальнейшего анализа.

In [4]:
# Удаляем ненужный столбец Unnamed: 0:
df.drop(columns=['Unnamed: 0'], inplace=True)



In [5]:
# Убираем возможные лишние пробелы из названий городов
df['city'] = df['city'].str.strip()

In [6]:
# Смотрим еще раз датасет
display(df)

,city,puid,hours
0,Москва,9668,26.167776
1,Москва,16598,82.111217
2,Москва,80401,4.656906
3,Москва,140205,1.840556
4,Москва,248755,151.326434
...,...,...,...
8779,Санкт-Петербург,1130000028554332,4.107774
8780,Санкт-Петербург,1130000030307246,45.069222
8781,Санкт-Петербург,1130000038726322,0.211944
8782,Санкт-Петербург,1130000047892100,4.311841


### 2. Проверка гипотезы в Python

Гипотеза звучит так: пользователи из Санкт-Петербурга проводят в среднем больше времени за чтением и прослушиванием книг в приложении, чем пользователи из Москвы. Попробуйте статистически это доказать, используя одностороннюю проверку гипотезы с двумя выборками:

- Нулевая гипотеза H₀: Средняя активность пользователей в часах в двух группах (Москва и Санкт-Петербург) не различается.

- Альтернативная гипотеза H₁: Средняя активность пользователей в Санкт-Петербурге больше, и это различие статистически значимо.

<b>Формулировка гипотез:</b>

Гипотеза H0: Нет разницы в средней активности пользователей из Москвы и Санкт-Петербурга.

Гипотеза H1: Пользователи из Санкт-Петербурга проводят больше времени за чтением и прослушиванием книг в приложении.

<b>Проведение теста:</b>

Используется критерий Стьюдента для несвязанных выборок с односторонней альтернативой. Уровень значимости традиционно принимаем равным α=0.05.

In [7]:
# Проверяем на наличие дуликатов
duplicate_puids = df.duplicated(subset='puid', keep=False).sum()
print(f'Количество дублирующихся идентификаторов пользователей: {duplicate_puids}')

Количество дублирующихся идентификаторов пользователей: 488


In [8]:
# Удаляем дубликаты и заново проверяем количество записей 
df_cleaned = df.drop_duplicates(subset=['puid'])
duplicate_puids_after_cleanup = df_cleaned.duplicated(subset='puid', keep=False).sum()
print(f'Количество дублирующихся идентификаторов пользователей после очистки: {duplicate_puids_after_cleanup}')

Количество дублирующихся идентификаторов пользователей после очистки: 0


In [9]:
# Разделим данные на 2 группы и оцениваем размер выборок
msk_group = df.query("city == 'Москва'")
spb_group = df.query("city == 'Санкт-Петербург'")
print("Размер выборки пользователей из Москвы:", len(msk_group))
print("Размер выборки пользователей из Санкт-Петербурга:", len(spb_group))

Размер выборки пользователей из Москвы: 6234
Размер выборки пользователей из Санкт-Петербурга: 2550


In [10]:
# Посчитаем средние часы активности и стандартные отклонения для обеих групп:
mean_msk = msk_group['hours'].mean()
std_msk = msk_group['hours'].std()

mean_spb = spb_group['hours'].mean()
std_spb = spb_group['hours'].std()

print(f"Средняя активность пользователей в Москве: {mean_msk:.2f}, стандартное отклонение: {std_msk:.2f}")
print(f"Средняя активность пользователей в СПб: {mean_spb:.2f}, стандартное отклонение: {std_spb:.2f}")

Средняя активность пользователей в Москве: 10.88, стандартное отклонение: 36.85
Средняя активность пользователей в СПб: 11.59, стандартное отклонение: 39.70


Выводы по статистике:

Анализ показал, что средняя активность пользователей незначительно отличается:

Москва: 10.88 ч

Санкт-Петербург: 11.59 ч

Стандартные отклонения высокие (36.85 и 39.70 соответственно), что подтверждает большую вариативность активности. 
Эти особенности предполагают использование <b> критерия Стьюдента с учётом неравенства дисперсий </b> для оценки статистической значимости выявленных различий.

In [11]:
# Выполняем односторонний t-тест для независимых выборок
result = stats.ttest_ind(spb_group['hours'], msk_group['hours'], equal_var=False, alternative="greater")  
p_value = result.pvalue
print(f'p-значение: {p_value:.4f}')

# Интерпретация полученного p-значения
if p_value > 0.05:
    interpretation = (
        f"Поскольку полученное p-значение ({p_value:.4f}) больше уровня значимости "
        "(обычно равного 0.05), мы не можем отвергнуть нулевую гипотезу. Это означает, что "
        "нет достаточных доказательств того, что сотрудники из Санкт-Петербурга работают больше, "
        "чем сотрудники из Москвы. Возможно, требуются дополнительные исследования или изменения "
        "бизнес-процессов."
    )
else:
    interpretation = (
        f"Поскольку полученное p-значение ({p_value:.4f}) меньше уровня значимости "
        "(обычно равного 0.05), мы отвергаем нулевую гипотезу. Это позволяет нам утверждать, что "
        "есть статистически значимая разница между средними рабочими часами сотрудников из "
        "Санкт-Петербурга и Москвы. Сотрудники из Санкт-Петербурга работают значительно больше."
    )

# Печать полного отчета
print("\nИнтерпретация результата:")
print(interpretation)

p-значение: 0.2182

Интерпретация результата:
Поскольку полученное p-значение (0.2182) больше уровня значимости (обычно равного 0.05), мы не можем отвергнуть нулевую гипотезу. Это означает, что нет достаточных доказательств того, что сотрудники из Санкт-Петербурга работают больше, чем сотрудники из Москвы. Возможно, требуются дополнительные исследования или изменения бизнес-процессов.


### 3. Аналитическая записка

<b>Аналитическая записка</b>

<b>Тип t-теста:</b> Односторонний независимый t-тест для двух выборок (Welch's t-test).

<b> Уровень значимости:</b>  
α=0.05

<b> Полученное p-значение:</b>  
0.2182

<b> Интерпретация результатов:</b>  Поскольку p-значение (
0.2182) больше установленного уровня значимости (α=0.05), мы не можем отвергнуть нулевую гипотезу. Следовательно, нет достаточных оснований утверждать, что среднее время активности пользователей из Санкт-Петербурга существенно больше, чем у пользователей из Москвы.

<b> Возможные объяснения отсутствия значимых различий:</b> 

В целом, поведение пользователей в обоих городах схожее, и среда проживания оказывает незначительное влияние на их предпочтения.
Возможно, выборка недостаточно репрезентативна, либо существуют внешние факторы, маскирующие реальные различия в потреблении контента.

Таким образом, имеющиеся данные не позволяют сделать уверенный вывод о превосходстве активности пользователей из Санкт-Петербурга над активностью москвичей.

----

## **Часть 2.** Анализ результатов A/B-тестирования

##  Цель исследования



Оценка эффективности новой версии интерфейса интернет-магазина BitMotion Kit путем анализа результатов проведенного A/B-теста.

### Загрузка данных


In [12]:
participants = pd.read_csv('https://code.s3.yandex.net/datasets/ab_test_participants.csv')
events = pd.read_csv('https://code.s3.yandex.net/datasets/ab_test_events.zip',
                     parse_dates=['event_dt'], low_memory=False)

In [13]:
display(participants.head())
participants.info()

,user_id,group,ab_test,device
0,0002CE61FF2C4011,B,interface_eu_test,Mac
1,001064FEAAB631A1,B,recommender_system_test,Android
2,001064FEAAB631A1,A,interface_eu_test,Android
3,0010A1C096941592,A,recommender_system_test,Android
4,001E72F50D1C48FA,A,interface_eu_test,Mac


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14525 entries, 0 to 14524
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  14525 non-null  object
 1   group    14525 non-null  object
 2   ab_test  14525 non-null  object
 3   device   14525 non-null  object
dtypes: object(4)
memory usage: 454.0+ KB


<b>Структура и характеристики датасета PARTICIPANTS: </b>

Всего записей: 14525

Количество столбцов: 4

Тип данных всех колонок: объект (object)

Пропущенных значений нет.

Датасет выглядит чистым и структурированным. Все колонки заполнены, отсутствуют пропуски. Строки соответствуют каждому участнику эксперимента с указанием типа устройства и проведённого А/Б теста. Проблем с качеством данных на данном этапе не выявлено.

In [14]:
display(events.head())
events.info()

,user_id,event_dt,event_name,details
0,GLOBAL,2020-12-01 00:00:00,End of Black Friday Ads Campaign,ZONE_CODE15
1,CCBE9E7E99F94A08,2020-12-01 00:00:11,registration,0.0
2,GLOBAL,2020-12-01 00:00:25,product_page,NaN
3,CCBE9E7E99F94A08,2020-12-01 00:00:33,login,NaN
4,CCBE9E7E99F94A08,2020-12-01 00:00:52,product_page,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 787286 entries, 0 to 787285
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   user_id     787286 non-null  object        
 1   event_dt    787286 non-null  datetime64[ns]
 2   event_name  787286 non-null  object        
 3   details     249022 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 24.0+ MB


<b> Cтруктура и характеристики датасета EVENTS: </b>

Всего записей: 787286

Количество столбцов: 4

Тип данных: datetime64[ns] (для event_dt), остальные объекты (object)

Значительное количество пропущенных данных в колонке details: 538264 строки содержат пустые значения.

Присутствуют странные записи с идентификатором "GLOBAL" в колонке user_id.


Записи с идентификатором "GLOBAL":Такие записи скорее всего являются общесистемными событиями, не относящимися к действиям конкретных пользователей. Их наличие может повлиять на дальнейшие расчёты и анализ поведения пользователей. Эти строки желательно исключить перед началом основной части анализа.

Пропуски в колонке "details":Поскольку эта колонка является необязательной и содержит много пропусков, её присутствие может затруднить анализ поведения пользователей. Нужно решить, как поступить с этими пропусками (например, оставить пустой, заполнить нулями или удалить).

Тип данных колонки "event_dt":Колонка имеет правильный тип данных datetime64[ns]. Она готова к дальнейшей обработке временных интервалов и событий.

<b>Следующие шаги: </b>

Исключаем строки с "GLOBAL" в user_id, поскольку они относятся к глобальным событиям и могут исказить картину действий реальных пользователей.

Удаляем колонку details.


In [15]:
# Шаг 1: удаление строк с идентификатором "GLOBAL"
events = events.query('user_id != "GLOBAL"')

# Шаг 2: удаление колонки "details"
events.drop(columns=['details'], inplace=True)

# Просмотр первых нескольких строк очищенного датасета
print(events.head())

            user_id            event_dt    event_name
1  CCBE9E7E99F94A08 2020-12-01 00:00:11  registration
3  CCBE9E7E99F94A08 2020-12-01 00:00:33         login
4  CCBE9E7E99F94A08 2020-12-01 00:00:52  product_page
5  AA346F4D22148024 2020-12-01 00:01:46  registration
6  7EF01D0E72AF449D 2020-12-01 00:02:06  registration


### По таблице `ab_test_participants` оцениваем корректность проведения теста:

   3\.1 Выделяем пользователей, участвующих в тесте, и проверяем:

   - соответствие требованиям технического задания,

   - равномерность распределения пользователей по группам теста,

   - отсутствие пересечений с конкурирующим тестом (нет пользователей, участвующих одновременно в двух тестовых группах).

In [16]:
# Посчитаем долю пользователей в каждой группе
group_counts = participants['group'].value_counts(normalize=True)
print(group_counts)

A    0.559725
B    0.440275
Name: group, dtype: float64


In [17]:
group_counts = participants['group'].value_counts()
print(group_counts)

A    8130
B    6395
Name: group, dtype: int64


In [18]:
# Проверяем пересечения с конкурирующим тестом
duplicate_users = participants.groupby(['user_id']).size().reset_index(name='count')
duplicate_tests = duplicate_users[duplicate_users['count'] > 1]['user_id']
if len(duplicate_tests) > 0:
    print(f"Пользователи участвуют в нескольких тестах:\n{duplicate_tests}")
else:
    print("Нет пользователей, участвующих в нескольких тестах.")

Пользователи участвуют в нескольких тестах:
1        001064FEAAB631A1
8        00341D8401F0F665
23       0082295A41A867B5
38       00E68F103C66C1F7
41       00EFA157F7B6E1C4
               ...       
13576    FEA0C585A53E7027
13582    FEC0BCA6C323872F
13605    FF2174A1AA0EAD20
13610    FF44696E39039D29
13636    FFF28D02B1EACBE1
Name: user_id, Length: 887, dtype: object


In [19]:
# Удаляем дублирующиеся записи (оставляем первую встречу)
clean_participants = participants.drop_duplicates(subset='user_id', keep='first')

# Перепроверяем наличие повторяющихся пользователей
new_duplicate_users = clean_participants.groupby(['user_id']).size().reset_index(name='count')
new_duplicate_tests = new_duplicate_users[new_duplicate_users['count'] > 1]['user_id']
if len(new_duplicate_tests) > 0:
    print(f"Остались пользователи, участвующие в нескольких тестах:\n{new_duplicate_tests}")
else:
    print("Все пользователи теперь уникальны.")

Все пользователи теперь уникальны.


### Анализируем данные о пользовательской активности по таблице `ab_test_events`:

- оставляем только события, связанные с участвующими в изучаемом тесте пользователями;

In [20]:
# Оставляем только тестируемых юзеров
test_user_ids = participants['user_id'].unique()

# Фильтруем по событию, оставляя только тестируемых юзеров
filtered_events = events[events['user_id'].isin(test_user_ids)]

display(filtered_events)

,user_id,event_dt,event_name
64672,5F506CEBEDC05D30,2020-12-06 14:10:01,registration
64946,51278A006E918D97,2020-12-06 14:37:25,registration
66585,A0C1E8EFAD874D8B,2020-12-06 17:20:22,registration
67873,275A8D6254ACF530,2020-12-06 19:36:54,registration
67930,0B704EB2DC7FCA4B,2020-12-06 19:42:20,registration
...,...,...,...
777488,F80C9BDDEA02E53C,2020-12-30 10:03:51,purchase
777489,F80C9BDDEA02E53C,2020-12-30 10:03:52,product_cart
778138,6181F3835EBE66BF,2020-12-30 12:10:39,product_cart
778369,DD4352CDCF8C3D57,2020-12-30 12:47:46,product_page


- определяем горизонт анализа: рассчитываем время (лайфтайм) совершения события пользователем после регистрации и оставляем только те события, которые были выполнены в течение первых семи дней с момента регистрации;

In [21]:
# Получаем серию дат регистрации для каждого пользователя
registrations = filtered_events.query("event_name == 'registration'")[['user_id', 'event_dt']]
registrations.rename(columns={'event_dt': 'registration_date'}, inplace=True)

# Объединяем с основными событиями, чтобы иметь доступ к дате регистрации
merged_data = filtered_events.merge(registrations, on='user_id', how='left')

In [22]:
# Преобразование временных типов
merged_data['event_dt'] = pd.to_datetime(merged_data['event_dt'])
merged_data['registration_date'] = pd.to_datetime(merged_data['registration_date'])

# Создаем новый столбец с временным промежутком
merged_data['days_since_registration'] = (merged_data['event_dt'] - merged_data['registration_date']).dt.days

In [32]:
# Остаемся только с событиями, выполненными в течение первых семи дней
final_filtered_data = merged_data[merged_data['days_since_registration'] < 8]

# Cмотрим получившуюся выборку
display(final_filtered_data)

,user_id,event_dt,event_name,registration_date,days_since_registration
0,5F506CEBEDC05D30,2020-12-06 14:10:01,registration,2020-12-06 14:10:01,0
1,51278A006E918D97,2020-12-06 14:37:25,registration,2020-12-06 14:37:25,0
2,A0C1E8EFAD874D8B,2020-12-06 17:20:22,registration,2020-12-06 17:20:22,0
3,275A8D6254ACF530,2020-12-06 19:36:54,registration,2020-12-06 19:36:54,0
4,0B704EB2DC7FCA4B,2020-12-06 19:42:20,registration,2020-12-06 19:42:20,0
...,...,...,...,...,...
98629,D4E530F6595A05A3,2020-12-29 23:39:54,product_cart,2020-12-22 06:00:57,7
98631,31DBDCA380DD035F,2020-12-29 23:41:16,product_page,2020-12-23 18:11:40,6
98638,E6D34EE376AADC42,2020-12-29 23:45:33,product_cart,2020-12-22 02:50:40,7
98639,E6D34EE376AADC42,2020-12-29 23:46:13,product_cart,2020-12-22 02:50:40,7


Оцениваем достаточность выборки для получения статистически значимых результатов A/B-теста. Заданные параметры:

- базовый показатель конверсии — 30%,

- мощность теста — 80%,

- достоверность теста — 95%.

In [25]:
base_conversion_rate = 0.30          
power = 0.80                          
confidence_level = 0.95                
expected_min_change = 0.05            

# Стандартные значения квантилей нормального распределения
z_alpha_half = norm.ppf((1 + confidence_level)/2)         
z_beta = norm.ppf(power)                                    

# Расчёт необходимой минимальной выборки на одну группу
def calculate_sample_size(base_conv_rate, expected_min_change, z_alpha_half, z_beta):
    sigma_squared = base_conv_rate * (1 - base_conv_rate)    
    delta = base_conv_rate * expected_min_change              
    
    sample_size_per_group = ((sigma_squared / delta**2) *
                             (z_alpha_half**2 + z_beta**2))
    
    return int(np.ceil(sample_size_per_group))                

# Минимально необходимая выборка на одну группу
min_sample_size_per_group = calculate_sample_size(
    base_conversion_rate, expected_min_change, z_alpha_half, z_beta
)

# Печать результата
print(f"Минимально необходимое количество пользователей в одной группе: {min_sample_size_per_group}")

# Допустим, общая выборка делится поровну на две группы
current_total_sample_size = 14525 // 2 

# Проверка достаточности вашей выборки
if current_total_sample_size >= min_sample_size_per_group:
    print("Вашей выборки достаточно для статистически значимых результатов.")
else:
    print("Вашей выборки недостаточно. Увеличьте объём выборки.")

Минимально необходимое количество пользователей в одной группе: 4247
Вашей выборки достаточно для статистически значимых результатов.


- рассчитываем для каждой группы количество посетителей, сделавших покупку, и общее количество посетителей.

In [26]:
# Фильтруем строки, содержащие событие покупки ("purchase") и добавляем группу участника
purchases_with_groups = final_filtered_data.merge(participants[["user_id", "group"]], on="user_id", how="inner").query("event_name == 'purchase'")

# Теперь подсчитываем уникальные покупки по каждой группе
buyers_per_group = purchases_with_groups.groupby('group')['user_id'].nunique()

# Общее число уникальных посетителей по группам
visitors_per_group = final_filtered_data.merge(participants[["user_id", "group"]], on="user_id", how="inner").groupby('group')['user_id'].nunique()

# Собираем результаты вместе
result = pd.DataFrame({
    'total_visitors': visitors_per_group,
    'buyers': buyers_per_group
})

# Рассчитываем коэффициент конверсии
result['conversion_rate'] = result['buyers'] / result['total_visitors']

print(result)

       total_visitors  buyers  conversion_rate
group                                         
A                7805    2230         0.285714
B                6279    1885         0.300207


Анализируя представленные данные, можно сделать следующий предварительный вывод:

В контрольной группе (A) из 7805 пользователей совершили покупку 2230 человек, что дало коэффициент конверсии примерно 28.57%.
В тестовой группе (B) из 6279 пользователей покупку совершили 1885 человек, что привело к коэффициенту конверсии примерно 30.02%.
Хотя тестовая группа имеет меньше пользователей, она показала чуть лучший коэффициент конверсии (+1.45%), что предполагает некоторое положительное влияние внесенных изменений на поведение пользователей. Однако, изначально ожидалось повышение коэффициента конверсии на 3 процентных пункта, однако фактический прирост составил лишь 1.45%. Это означает, что цель достигнута не была.

Тем не менее, несмотря на благоприятные тенденции, необходимы дополнительные меры предосторожности и статистические тесты, чтобы гарантировать значимость полученных результатов. Ключевыми вопросами остаются:

Достаточно ли велика выборка для подтверждения статистической значимости?
Может ли эта небольшая разница быть следствием случайности?
Есть ли риски неравномерного распределения характеристик пользователей между группами?
Таким образом, пока можно осторожно предположить, что обновления оказали позитивное влияние на поведение пользователей, повысив конверсию. Однако точный вывод потребует дополнительной верификации с применением методов статистического анализа.

### Проводим оценку результатов A/B-тестирования:

<b>Формулировка гипотез </b>

Нулевая гипотеза (H0): Нет значимых различий в уровне конверсии между группой А и группой Б.

Альтернативная гипотеза (H1): Имеются значительные различия в уровне конверсии между двумя группами.


In [27]:
# Определение общих оценок
n_A = result.loc['A', 'total_visitors']
x_A = result.loc['A', 'buyers']
p_A = x_A / n_A

n_B = result.loc['B', 'total_visitors']
x_B = result.loc['B', 'buyers']
p_B = x_B / n_B

# Средневзвешенный коэффициент конверсии
p_pool = (x_A + x_B) / (n_A + n_B)

# Стандартная ошибка
se = np.sqrt(p_pool * (1 - p_pool) * (1 / n_A + 1 / n_B))

# Z-статистика
z_statistic = (p_A - p_B) / se

# Уровень значимости
alpha = 0.05

# Критическое значение для двустороннего теста
critical_value = norm.ppf(1 - alpha / 2)

# Проверка гипотез
if abs(z_statistic) > critical_value:
    conclusion = f"Нулевая гипотеза отклоняется. Наблюдается значимая разница в конверсии."
else:
    conclusion = f"Нулевая гипотеза принимается. Значимой разницы в конверсии не обнаружено."

# Печать результатов
print(f"Z-статистика: {z_statistic:.3f}, критическое значение: {critical_value:.3f}.")
print(conclusion)

Z-статистика: -1.880, критическое значение: 1.960.
Нулевая гипотеза принимается. Значимой разницы в конверсии не обнаружено.


<b>Цель A/B-тестирования состояла в проверке влияния новой версии интерфейса на поведение пользователей интернет-магазина. Было проведено сравнение двух групп: контрольной (группа A) и тестовой (группа B), с целью выяснить, улучшилась ли конверсия пользователей благодаря изменениям интерфейса.</b>

<b>Гипотеза:</b>

Нулевая гипотеза: изменения интерфейса не оказывают значимого влияния на конверсию пользователей.

Альтернативная гипотеза: новая версия интерфейса приводит к значимым изменениям в конверсии.

<b>Анализ данных:</b>

Группа A (контрольная группа):
Всего пользователей: 7805
Совершили покупку: 2230
Коэффициент конверсии: 28.57%

Группа B (тестовая группа):
Всего пользователей: 6279
Совершили покупку: 1885
Коэффициент конверсии: 30.02%

<b>Статистический анализ:</b>

Был проведен z-тест для пропорций, чтобы проверить, отличаются ли коэффициенты конверсии в группах A и B. 

<b>Основной результат:</b>

Z-статистика составила примерно −1.880, что меньше, чем критическое значение для уровня значимости 
α=0.05 (±1.96). 

<b>Интерпретация:</b>

Так как модуль z-статистики меньше критического значения (1.880 < 1.960), нулевая гипотеза НЕ ОТКАЗЫВАЕТСЯ. То есть полученные различия в конверсиях между контрольной и тестовой группами НЕ ЯВЛЯЮТСЯ СТАТИСТИЧЕСКИ ЗНАЧИМЫМИ.

<b>Заключение:</b>

Проведенное A/B-тестирование не позволило обнаружить значимые отличия в поведении пользователей контрольной и тестовой групп. Новая версия интерфейса не привела к ощутимому повышению конверсии, следовательно, нельзя утверждать, что сделанные изменения положительно повлияли на покупательское поведение пользователей. Требуются дальнейшие исследования или улучшение дизайна интерфейса для достижения желаемого роста конверсии.